# Greedy (demo)

In [45]:
import math
import time
import os
import csv
import re
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd

In [46]:
# convert txt to csv

# input test demo file and convert to csv
input_file = "HMM_Algeciras.txt"
output_file = "HMM_Algeciras.csv"

pattern = r"\[(\d+),(\d+)\], \{(\d+)\}, (.*)"

with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["row", "column", "weight", "description"])

    for line in infile:
        line = line.strip()
        match = re.match(pattern, line)
        if match:
            row, col, weight, desc = match.groups()
            writer.writerow([row, col, weight, desc])

print(f"Done: {output_file}")


Done: HMM_Algeciras.csv


1. Define the representation and initial state, and import and define dataset:

    - Convert given dataset to 2D numpy array
    - Dataset generate by us in additional to given test set
    
*Note that crane located at [0, 8] (by slides, just above [1, 8])*

In [47]:
# This is for given test set only because row 1 start from bottom- need to flip it during computing.
# Store mainfest: NAN as -1, 0 as empty, >0 as occupied, number are weight of the container
def create_grid(path):
    df = pd.read_csv(path)
    nrows = int(df["row"].max()) + 1
    ncols = int(df["column"].max())
    grid = np.zeros((nrows, ncols), dtype=int)

    for _, rec in df.iterrows():
        r = nrows - int(rec["row"])
        c = int(rec["column"]) - 1
        desc = str(rec["description"]).strip().upper()

        if desc == "NAN":
            grid[r, c] = -1
        elif desc == "UNUSED":
            grid[r, c] = 0
        else:
            grid[r, c] = int(rec["weight"])
    return grid

test_demo = create_grid(output_file)
test_demo

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ -1,   0,   0,   0, 123, 300,   0,   0,   0,   0,   0,  -1]])

At this points, all dataset are store in 2D numpy array format, 9 by 12

2. Define goal state

In [48]:
def left_weight(grid):
    return int(np.clip(grid[1:9, 0:6], 0, None).sum())

def right_weight(grid):
    return int(np.clip(grid[1:9, 6:12], 0, None).sum())

def which_side_is_heavier(grid):
    left = left_weight(grid)
    right = right_weight(grid)
    return (0, 6) if left > right else (6, 12)

def goal_test(grid, tol=0.10):
    total = left_weight(grid) + right_weight(grid)
    return True if total == 0 else abs(left_weight(grid) - right_weight(grid)) <= tol * total

# determine if the grid is balanceable
def is_balanceable(grid):
    container = []

    # loop all grid(each col and row), add each non zero value to container list
    for row in grid:
        for col in row:
            if col != 0 and col != -1:
                container.append(col)

    # check if the container list is balanceable
    total = sum(container)
    threshold = total * 0.1
    n = len(container)

    # use brute force to check all combination since max 16 containers
    for mask in range(1, (1 << n)):
        sum1 = 0
        for i in range(n):
            if mask & (1 << i):
                sum1 += container[i]
        
        sum2 = total - sum1
        if abs(sum1 - sum2) <= threshold:
            return True
    
    return False

# goal_test(grid4)

In [49]:
# def get_set_data(data):
#     left = left_weight(data)
#     right = right_weight(data)
#     goal = goal_test(data)
#     total = left + right
#     needed = math.ceil((abs(left-right) - 0.1*total) / 2)

#     print("Left: ", left, "\nRight: ", right)
#     print("Total: ", total)
#     print("Diff: ", abs(left-right))
#     print("Goal: ", goal)
#     print("Target: ", 0.1*total) # weight diff small or equal to this number
#     print("Needed: ", needed)

# get_set_data(grid7)

3. Define the operators (What to move)

In [50]:
def determine_container_to_move(grid):
    # compute which side is heavier
    left_sum = left_weight(grid)
    right_sum = right_weight(grid)

    low, high = (0, 6) if left_sum > right_sum else (6, 12)
    
    # comopute the needed weight to move
    total = left_sum + right_sum
    diff = abs(left_sum - right_sum)
    target = total * 0.1
    needed = math.ceil((diff - target) / 2)
    
    # find the container to move
    candidates = [((row, col), int(grid[row, col]))
                    for row in range(grid.shape[0])
                    for col in range(low, high)
                    if int(grid[row, col]) > 0]
    
    # a. check if can do done by move single container
    needed_upper = math.floor((diff + target) / 2)

    for pos, weight in candidates:
        if needed <= weight <= needed_upper:
            return [(pos, weight)], needed
        # else: continue
    
    # b. otherwise, try combinations containers by getting the very top container
    top_sorted = sorted(candidates, key=lambda x: x[0][0])
    current_combination = []
    current_weight = 0

    for pos, weight in top_sorted:
        current_combination.append((pos, weight))
        current_weight += weight
        if current_weight >= needed:
            return current_combination, needed
    
    return candidates, needed

# candidates, needed = determine_container_to_move(grid4)
# candidates, needed


After determine which container need to move, now it's time to move them (greedy move):

4. Search algorithm (Actual Operation)

    - If there are a container above it, move it into nearest column on the same side.
    - Slide horizontally to the right: grid to the right might be blocked, move up until right cell is empty; then move right; repeat if same silution happen. 
    - Drop down. Once on the target side, drop it down. To save steps, can drop on columns that nearest the border. 

In [51]:
# check container above it
def is_container_above(grid, row, col):
    return row > 0 and grid[row - 1, col] > 0

# handle vertical move in a column(where to drop in this column)
def find_drop_in_column(grid, col):
    last = grid.shape[0] - 1
    if grid[last, col] == 0:
        return (last, col)
    for row in range(last - 1, 0, -1):
        if grid[row, col] == 0 and grid[row + 1, col] > 0:
            return (row, col)
    return None

# handle horizontal move in a column(which column should drop to)
def nearest_empty_column(grid, src_col, target_cols):
    available = []
    for c in target_cols:
        drop = find_drop_in_column(grid, c)
        if drop is not None:
            available.append((abs(c - src_col), drop))

    if not available:
        return None

    _, best_drop = min(available, key=lambda x: x[0])
    return best_drop

# find_drop_in_column(grid4, 0)
# nearest_empty_column(grid4, 0, range(11, 5, -1))


In [52]:
# helper function: find destination
def find_destination(grid, target_cols):
    last_row = grid.shape[0] - 1

    # a. last row first
    for col in target_cols:
        if grid[last_row, col] == 0:
            return (last_row, col)

    # b. if no bottom slots, stack upward
    for row in range(last_row - 1, 0, -1):
        for col in target_cols:
            if grid[row, col] == 0 and grid[row + 1, col] > 0:
                return (row, col)

    raise ValueError("No destination found")

# main function: move container to another side
def move_container_another_side(grid, container_to_move, log_print=print):

    move_count = 0

    # check which side is heavier
    heavier_side = which_side_is_heavier(grid)
    target_cols = range(11, 5, -1) if heavier_side == (0, 6) else range(0, 6)

    # crane location
    PARK = (0, 8)
    crane_row, crane_col = PARK

    # track steps
    per_container_steps = []
    total_steps = 0

    # start moving
    for (src_row, src_col), _  in container_to_move:

        # a. clear blockers on SAME side
        same_side = range(0, 6) if src_col < 6 else range(6, 12)
        while is_container_above(grid, src_row, src_col):
            blocker_row = src_row - 1

            # choose nearest same-side column with a legal drop spot
            stash_cols = [c for c in same_side if c != src_col]
            drop = nearest_empty_column(grid, src_col, stash_cols)
            if drop is None:
                raise ValueError("No place to stash a blocker on this side")

            drb, dcb = drop

            leg1 = abs(crane_row - blocker_row) + abs(crane_col - src_col)
            leg2 = abs(drb - blocker_row) + abs(dcb - src_col)

            grid[drb, dcb] = grid[blocker_row, src_col]
            grid[blocker_row, src_col] = 0

            crane_row, crane_col = drb, dcb
            step_block = leg1 + leg2
            total_steps += step_block
            per_container_steps.append(((blocker_row, src_col), (drb, dcb), step_block))
        
        destination = find_destination(grid, target_cols)
        
        dr, dc = destination
        # move 1: crane -> container(diatance from current crane pos to next containers pos)
        move1 = abs(crane_row - src_row) + abs(crane_col - src_col)
        # move 2: container -> destination(distance to carry container from it source to target destination)
        move2 = abs(dr - src_row) + abs(dc - src_col)

        # perform move
        grid[dr, dc] = grid[src_row, src_col]
        grid[src_row, src_col] = 0

        crane_row, crane_col = dr, dc  # update crane pos

        # update steps
        steps = move1 + move2
        per_container_steps.append(((src_row, src_col), (dr, dc), steps))
        total_steps += steps

        move_count += 1
        log_print(f"Move {move_count}: [{src_row}, {src_col}] -> [{dr}, {dc}], Steps(minutes): {steps}")

    # final move: crane -> park
    total_steps += abs(crane_row - PARK[0]) + abs(crane_col - PARK[1])


    return grid, per_container_steps, total_steps
        
# test
# grid, per_container_steps, total_steps = move_container_another_side(grid4, candidates)
# print("\nGrid: \n", grid)
# print("\nMoves: \n", per_container_steps)
# print(f"\nTotal steps: {total_steps}")

In [53]:
# ask user to make a comment and print in log file
comment = input("Any comments about this run? (press Enter to skip): ")
comment = comment.strip() if comment.strip() else "N/A"
comment

'This is a test comments'

In [54]:
def main(grid, grid_name="grid", comment="N/A"):
    
    # generate log: {txt prefix}MM_DD_YYYY_HHMM_gridN.txt
    now = datetime.now()
    timestamp = now.strftime("%m_%d_%Y_%H%M")
    # output log name should be same with txt prefix
    prefix = os.path.splitext(input_file)[0]
    log_filename = f"{prefix}{timestamp}.txt"
    
    # write log file in the same folder
    log_file = log_filename
    
    with open(log_file, "w") as f:
        def log_print(*args, **kwargs):
            # Add timestamp to each log entry
            timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            msg = " ".join(str(arg) for arg in args)
            log_entry = f"[{timestamp_str}] {msg}"
            print(*args, **kwargs)
            f.write(log_entry + "\n")
            f.flush()

        log_print("Program starts. ")
        container_count = np.sum((grid > 0))
        log_print(f"File open, this is {grid_name}. There are total of {container_count} containers on the ship.")

        # 1. check if goal state
        if goal_test(grid):
            log_print("This mainfest is already balanced, no action needed.")
            total_steps = 0
            log_print(f"A comment was written to the log : {comment}")
            log_print(f"Program ends. Computation time: {total_steps} minutes")
            return None, [], 0

        # 2. check if balanceable
        elif not is_balanceable(grid):
            log_print("This mainfest is not balanceable, no action needed")
            total_steps = 0
            log_print(f"A comment was written to the log : {comment}")
            log_print(f"Program ends. Computation time: {total_steps} minutes")
            return None, [], 0

        # 3. determine which container to move
        candidates, needed = determine_container_to_move(grid)

        # 4. move container to another side
        grid, per_container_steps, total_steps = move_container_another_side(grid, candidates, log_print)
        log_print(f"Finished a cycle. Total steps: {total_steps}")
        # log_print(f"Moves: {per_container_steps}")
        log_print(f"Balanced Grid: \n{grid}")
        
        # End timing and log (1 move = 1 minute)
        log_print("File was written to destop.")
        # print comments
        log_print(f"A comment was written to the log : {comment}")

        log_print(f"Program ends. Computation time: {total_steps} minutes")

    return grid, per_container_steps, total_steps


In [55]:
# Process test_demo and generate log file
print("Processing test_demo...")
result = main(test_demo, grid_name="test_demo", comment=comment)
print(f"Completed. Log file saved in current folder.")

Processing test_demo...
Program starts. 
File open, this is test_demo. There are total of 2 containers on the ship.
This mainfest is not balanceable, no action needed
A comment was written to the log : This is a test comments
Program ends. Computation time: 0 minutes
Completed. Log file saved in current folder.
